In [5]:
#Paquetes necesarios
import cv2
import numpy as np
from deepface import DeepFace

In [18]:
def overlay_image_alpha(img, overlay, position, alpha_mask):
    """Superpone una imagen `overlay` en la posición especificada sobre la `img`."""
    x, y = position
    h, w = overlay.shape[:2]

    # Asegurar que el overlay encaje dentro de los límites de `img`
    if y < 0:  # Si el overlay se sale por arriba, recortamos
        overlay = overlay[-y:, :, :]
        alpha_mask = alpha_mask[-y:, :]
        y = 0
    if x < 0:  # Si el overlay se sale por la izquierda, recortamos
        overlay = overlay[:, -x:, :]
        alpha_mask = alpha_mask[:, -x:]
        x = 0
    if y + h > img.shape[0]:  # Si el overlay se sale por abajo, recortamos
        h = img.shape[0] - y
        overlay = overlay[:h, :, :]
        alpha_mask = alpha_mask[:h, :]
    if x + w > img.shape[1]:  # Si el overlay se sale por la derecha, recortamos
        w = img.shape[1] - x
        overlay = overlay[:, :w, :]
        alpha_mask = alpha_mask[:, :w]

    # Corta la región donde se colocará el overlay
    region = img[y:y+h, x:x+w]

    # Asegura que alpha_mask esté en el rango correcto
    alpha_mask = alpha_mask.astype(float) / 255.0

    # Combina el overlay con la región del frame usando el canal alfa
    for c in range(3):  # Para cada canal (RGB)
        region[:, :, c] = (1.0 - alpha_mask) * region[:, :, c] + alpha_mask * overlay[:, :, c]

    img[y:y+h, x:x+w] = region
    return img

In [19]:
# Configuración de la cámara y el modelo DNN para detección de rostros
dnn_model = "deploy.prototxt.txt"
dnn_weights = "res10_300x300_ssd_iter_140000.caffemodel"
net = cv2.dnn.readNetFromCaffe(dnn_model, dnn_weights)
video_capture = cv2.VideoCapture(0)

# Cargar gráficos del cabello y aura (asegúrate de que son PNG con canal alfa)
hair = cv2.imread('pelo_super_saiyan.png', cv2.IMREAD_UNCHANGED)
aura = cv2.imread('aura_super_saiyan.png', cv2.IMREAD_UNCHANGED)

# Procesar el video en tiempo real
while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # Detección de rostros
    h, w = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), [104.0, 177.0, 123.0], False, False)
    net.setInput(blob)
    detections = net.forward()

    # Para cada cara detectada
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (x, y, x2, y2) = box.astype("int")
            face_width = x2 - x
            face_height = y2 - y

            # Detectar emoción con DeepFace
            try:
                face_img = frame[y:y2, x:x2]  # Extrae el rostro detectado
                analysis = DeepFace.analyze(face_img, actions=['emotion'], enforce_detection=False)
                emotion = analysis[0]['dominant_emotion']

                # Activar efectos si la emoción es enfado
                if emotion == 'angry':
                    # Redimensionar gráficos de cabello y aura
                    hair_resized = cv2.resize(hair, (face_width, face_height))
                    aura_resized = cv2.resize(aura, (face_width * 2, face_height * 2))  # Aura puede ser más grande

                    # Extraer canal alfa como máscara
                    hair_alpha = hair_resized[:, :, 3]  # Canal alfa del cabello
                    aura_alpha = aura_resized[:, :, 3]  # Canal alfa del aura

                    # Superponer gráficos sobre el frame en posiciones específicas
                    frame = overlay_image_alpha(frame, hair_resized[:, :, :3], (x, y - face_height // 2), hair_alpha)
                    frame = overlay_image_alpha(frame, aura_resized[:, :, :3], (x - face_width // 2, y - face_height), aura_alpha)

            except Exception as e:
                print("Error en análisis de emociones:", e)

    cv2.imshow('Filtro Saiyan', frame)
    if cv2.waitKey(1) & 0xFF == 27:  # Salir con ESC
        break

# Liberar la cámara y cerrar ventanas
video_capture.release()
cv2.destroyAllWindows()

Error en análisis de emociones: operands could not be broadcast together with shapes (400,358) (480,358) 
Error en análisis de emociones: operands could not be broadcast together with shapes (403,358) (480,358) 
Error en análisis de emociones: operands could not be broadcast together with shapes (393,346) (480,346) 
Error en análisis de emociones: operands could not be broadcast together with shapes (391,342) (480,342) 
Error en análisis de emociones: operands could not be broadcast together with shapes (394,340) (480,340) 
Error en análisis de emociones: operands could not be broadcast together with shapes (395,336) (480,336) 
Error en análisis de emociones: operands could not be broadcast together with shapes (389,344) (480,344) 
Error en análisis de emociones: operands could not be broadcast together with shapes (394,346) (480,346) 
Error en análisis de emociones: operands could not be broadcast together with shapes (396,348) (480,348) 
Error en análisis de emociones: operands could